# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd

Прочитаем файл и сохраним его в переменной *df*.

In [2]:
df = pd.read_csv('/datasets/data.csv')

Выведем первые 10 строк таблицы.

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о данных таблицы *df*.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в таблице 12 столбцов, с разными типами данных. По 5 столбцов имеют типы данных *int64* и *object*, оставшиеся 2 столбца - *float64*.

Столбцы в таблице *df* и какую информацию они содержат:
* children — количество детей в семье;
* days_employed — общий трудовой стаж в днях;
* dob_years — возраст клиента в годах;
* education — уровень образования клиента;
* education_id — идентификатор уровня образования;
* family_status — семейное положение;
* family_status_id — идентификатор семейного положения;
* gender — пол клиента;
* income_type — тип занятости;
* debt — имел ли задолженность по возврату кредитов;
* total_income — ежемесячный доход;
* purpose — цель получения кредита.

**Вывод**

Каждая строка в данной таблице содержит определенную информацию о клиенте, а именно: его количество детей, общий трудовой стаж, возраст, уровень образования, семейное положение, пол, тип занятости, информацию о задолжностях по возврату кредитов, ежемесячный доход и цель получения кредита. 

Количество значений в столбцах *days_employed* и *total_income* отличаются от остальных. К тому же эти 2 столбца имеют одинаковое количество значений.

На данном этапе анализа можно выделить несколько проблем, которые нужно решить:
* пропуски в столбцах *days_employed* и *total_income*;
* отрицательные значения в столбце *days_employed*;
* преобразование типа *float64* в тип *int64* и его последующее округление в столбцах *total_income* и *days_employed* для удобства работы с информацией;
* значения с символами верхнего регистра в столбце *education*. 

В пропусках наблюдается некая закономерность, так как пропущенные значения в столбцах *days_employed* и *total_income* присутсвуют в одних и тех же строках, что, возможно, обусловлено технической ошибкой.

Возможно, в ходе дальнейшего анализа выявятся и другие проблемы.

Для исследования заданного вопроса нас интересуют столбцы *children*, *family_status*, *family_status_id* и *debt*.

## Шаг 2. Предобработка данных

### Обработка пропусков

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [5]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски выявились в столбцах *days_employed* и *total_income*. Оба столбца отображают количественные переменные, поэтому их пропуски мы будем заполнять характерными значениями. Для этого нам пригодятся среднее арифметическое и медиана.

Заполнять пропуски начнем со столбца *total_income*. Для начала проверим есть ли там отрицательные значения, как в столбце *days_employed*.

In [6]:
df['total_income'][df['total_income'] < 0].count()

0

Количество отрицательных равно 0. Значит можно смело искать среднее арифметическое и медиану значений. Чтобы все было по-честному сгруппируем полученные значения по типу занятости. 

In [7]:
total_income_grouped_mean = df.groupby('income_type')['total_income'].mean() 
display(total_income_grouped_mean)

total_income_grouped_median = df.groupby('income_type')['total_income'].median() 
print(total_income_grouped_median)

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


Значения, посчитанные через среднее арифметическое и медиану, отличаются некритично, поэтому при заполнении пропусков будем использовать среднее арифметическое.

Заполним пропуски в столбце *total_income* с группировкой по переменной *income_type*.

In [8]:
for type in df['income_type'].unique():
    mean_income = df.loc[df['income_type'] == type, 'total_income'].mean()
    df.loc[(df['total_income'].isna()) & (df['income_type'] == type), 'total_income'] = mean_income

Аналогичным способом заполним пропуски в столбце *days_employed*, но до этого мы должны проделать необходимые операции со значениями в этом столбце. Некоторые значения в столбце имеют отрицательное значение. Проверим есть ли какая-то зависимость от знака перед значением.

In [9]:
print(df['days_employed'][df['days_employed'] < 0].count())
df['days_employed'][df['days_employed'] < 0].head(20)

15906


0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
5      -926.185831
6     -2879.202052
7      -152.779569
8     -6929.865299
9     -2188.756445
10    -4171.483647
11     -792.701887
13    -1846.641941
14    -1844.956182
15     -972.364419
16    -1719.934226
17    -2369.999720
19   -10038.818549
20    -1311.604166
21     -253.685166
22    -1766.644138
Name: days_employed, dtype: float64

In [10]:
print(df['days_employed'][df['days_employed'] > 0].count())
df['days_employed'][df['days_employed'] > 0].head(20)

3445


4      340266.072047
18     400281.136913
24     338551.952911
25     363548.489348
30     335581.668515
35     394021.072184
50     353731.432338
56     370145.087237
71     338113.529892
78     359722.945074
86     383933.549664
87     337659.535004
88     373129.789654
98     364906.205736
99     346541.618895
101    391558.961849
104    366323.649429
119    382189.129056
128    379035.920619
129    361010.321350
Name: days_employed, dtype: float64

Положительные значения в столбце *days_employed* имеют неоправданно большие значения и к тому же составляют 16% от всех значений, значит просто удалить их нельзя.
Скорее всего значения со положительным знаком отображают общий трудовой стаж не в днях, а в часах.
Отобразим все положительные значения в днях. Для этого поделим каждое из них на 24, с помощью функции *hours_to_days*. После этого проверим результат.

In [11]:
def hours_to_days(data):
    if data['days_employed'] > 0:
        return (data['days_employed'] / 24)
    else:
        return data['days_employed']
df['days_employed'] = df.apply(hours_to_days, axis = 1)
df['days_employed'][df['days_employed'] > 0].head(20)

4      14177.753002
18     16678.380705
24     14106.331371
25     15147.853723
30     13982.569521
35     16417.544674
50     14738.809681
56     15422.711968
71     14088.063746
78     14988.456045
86     15997.231236
87     14069.147292
88     15547.074569
98     15204.425239
99     14439.234121
101    16314.956744
104    15263.485393
119    15924.547044
128    15793.163359
129    15042.096723
Name: days_employed, dtype: float64

Теперь все значения в столбце *days_employed* представлены в днях. Пришло время разобраться с отрицательными значениями. Отрицательными они получились скорее всего из-за того, что в таблицу вносили разность между временем вношения данных в таблицу и временем начала отстчета трудового стажа относительно UTC.

Теперь пришло время разобраться с отрицательными значениями.

In [12]:
df['days_employed'] = df['days_employed'].apply(abs)
df['days_employed'][df['days_employed'] < 0].count()

0

После успешной проверки на наличие отрицательных значений в столбце, наконец-то можно приступить к заполнению пропусков с группировкой по переменной *income_type*.

In [13]:
for type in df['income_type'].unique():
    mean_days = df.loc[df['income_type'] == type, 'days_employed'].mean()
    df.loc[(df['days_employed'].isna()) & (df['income_type'] == type), 'days_employed'] = mean_days

Проверим наличие пропусков в таблице.

In [14]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Мы окончательно избавились от пропусков.

**Вывод**

Пропуски в столбцах *days_employed* и *total_income* выявлялись в одинаковых строках, что говорит о системной ошибке при введении значений в таблицу. Столбец *days_employed* можно было просто удалить без обработки значений, так как информация из него не нужна для ответов на поставленные вопросы. 

### Замена типа данных

Значения в столбцах *days_employed* и *total_income* имеют тип *float64*. В данном случае этот тип выглядит непрезентабельно, поэтому придется заменить тип данных в этих столбцах на *int* с помощью метода *astype*.

In [15]:
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Со значениями в вышеописанных столбцах неудобно работать с изначальным типом *float64* и, так как в нашем случае этот тип не подходит данным значениям, мы перевели их в целое число.

### Обработка дубликатов

Для действенного поиска и дальнейшего удаления дубликатов необходимо привести все символы в столбцах с типом *object*, если это необходимо, к нижнему регистру. Проверим каждый такой столбец на количесвто уникальных значений.

In [16]:
display(df['education'].value_counts())
display(df['family_status'].value_counts())
display(df['gender'].value_counts())
display(df['income_type'].value_counts())
display(df['purpose'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Видно, что в столбце *education* необходимо применить понижение регистра символов.

In [17]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

В столбце *family_status* присутствует один вид значения *Не женат / не замужем* с символом верхнего регистра, но понижение регистра этого символа ничего кардинально не изменит, так как он по своему смыслу уникален, поэтому оставляем этот столбец без изменений.
Значения в столбце *income_type* при поиске дубликатов проблем не принесут. Аналогичная ситуация со столбцами *purpose* и *gender*, но в столбце *gender* выделяется значение *XNA*, которое встречается всего в одной строке. Мы можем либо удалить эту строку, так как это никак не отобразится на результате исследования, либо заменить его на более часто встречающееся *F*. Выберем второй вариант.

In [18]:
df['gender'] = df['gender'].replace('XNA', 'F')
display(df['gender'].value_counts())

F    14237
M     7288
Name: gender, dtype: int64

Теперь можно приступить к поиску дубликатов.

In [19]:
df.duplicated().sum()

71

Найден 71 дубликат. Удаляем их, без создавания столбцов со старыми значениями индексов.

In [20]:
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

**Вывод**

Дубликаты удалены. Возможно, появление дубликатов обусловлено тем, что данные собирались из разобщенных источников организации и в последствии были объеденены в одну базу данных. Также имеет место быть, что в этих источниках по разному велась информация, например, об образовании. К примеру: *среднее* или *СРЕДНЕЕ*. 

### Лемматизация

По столбцу *purpose* заметно, что клиенты по-разному описывают цели для получения кредита, но в общем итоге имеют ввиду похожие категории. Напрмер, *свадьба*, *на проведение свадьбы*, *сыграть свадьбу*. Эти значения можно привести к одной категории - *свадьба*. Для этого нужно провести лемматизацию всего столбца *purpose*.

Найдем наиболее встречающиеся ключевые слова в столбце. Для этого создадим новый столбец *key_purpose*, где будет находится лемма соответствующей строки, c помощью функции *lem_purpose* и посчитаем количество каждого из встречающихся слов в столбце, вызвав *Counter*.

In [21]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

def lem_purpose(purpose):
    lem = m.lemmatize(purpose)
    return lem

df['key_purpose'] = df['purpose'].apply(lem_purpose)
print(df['key_purpose'])

Counter(df['key_purpose'].sum())

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21449                  [операция,  , с,  , жилье, \n]
21450               [сделка,  , с,  , автомобиль, \n]
21451                              [недвижимость, \n]
21452    [на,  , покупка,  , свой,  , автомобиль, \n]
21453             [на,  , покупка,  , автомобиль, \n]
Name: key_purpose, Length: 21454, dtype: object


Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         '\n': 21454,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

Из полученных результатов можно выделить следующие категории:
* автомобиль;
* образование;
* свадьба;
* недвижимость(жилье).

За дальнейшей ненадобностью удалим столбец *key_purpose*.

In [22]:
df = df.drop('key_purpose', axis =1)

Применив функцию *main_to_common* добавим в таблицу новый столбец *common_purpose*, в котором все цели кредита заменены на их обобщенное описание.

In [23]:
from pymystem3 import Mystem
m = Mystem()
df['common_purpose'] = 0
def main_to_common(data):
    for i in range(0, len(data['common_purpose'])):
        lemma = m.lemmatize(data.loc[i, 'purpose'])
        if 'автомобиль' in lemma:
            data.loc[i, 'common_purpose'] = 'автомобиль'
        if 'образование' in lemma:
            data.loc[i, 'common_purpose'] = 'образование'
        if 'свадьба' in lemma:
            data.loc[i, 'common_purpose'] = 'свадьба'
        if 'недвижимость' in lemma or 'жилье' in lemma:
            data.loc[i, 'common_purpose'] = 'недвижимость'
    return data

df = main_to_common(df)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,common_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


**Вывод**

С помощью лемматизации мы упростили себе путь к категоризации данных.

### Категоризация данных

Для того, чтобы проще было ориентироваться в таблице создадим несколько "словарей". Сначала, создадим "словарь" *meaning_of_purpose*, где будут присутствовать столбцы *purpose* и *common_purpose*.
Удалим дубликаты сразу после создания таблицы *meaning_of_purpose*, чтобы привести ее к "словарному" виду.

In [24]:
meaning_of_purpose = df[['purpose', 'common_purpose']]
meaning_of_purpose = meaning_of_purpose.drop_duplicates().reset_index(drop = True)
meaning_of_purpose

,purpose,common_purpose
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,дополнительное образование,образование
3,сыграть свадьбу,свадьба
4,операции с жильем,недвижимость
5,образование,образование
6,на проведение свадьбы,свадьба
7,покупка жилья для семьи,недвижимость
8,покупка недвижимости,недвижимость
9,покупка коммерческой недвижимости,недвижимость


В следующем "словаре" *family_statuses* покажем какому семейнему статусу соответствует его *id*. Для этого объеденим столбцы *family_status* и *family_statuses_id*, и удалим дубликаты из них.

In [25]:
family_statuses = df[['family_status', 'family_status_id']]
family_statuses = family_statuses.drop_duplicates().reset_index(drop = True)
family_statuses

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Следующий "словарь" *education_type* будет похож на предыдущий, только уже с видом образования клиента и его *id*. Объединяем столбцы *education* и *education_id*.

In [26]:
education_type = df[['education', 'education_id']]
education_type = education_type.drop_duplicates().reset_index(drop = True)
education_type

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Так как нам надо будет ответить на вопрос о зависимости возврата кредита в срок от наличия детей, то классифицируем количество детей на группы.
Узнаем какие количества детей присутсвтуют в таблице.

In [27]:
df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В данных появились значения *-1* и *20*, которые мы пропустили при обработке пропусков и дубликатов. В будущем надо быть внимательнее. Заменим *-1* на *1*. Значение *20*, скорее всего, появилось из-за опечатки клиента при вводе данных, так как числа *2* и *0* близко находятся на *Numpad-e*. Мы не знаем сколько хотел ввести клиент (*2* или *0*), поэтому заменим *20* на медиану.

In [28]:
df['children'] = df['children'].replace(-1, 1)
df['children'].value_counts()

0     14091
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [29]:
median_children = df['children'].median()
df['children'] = df['children'].replace(20, median_children)
df['children'].value_counts()

0    14167
1     4855
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Пришло время категоризировать данные по зарплате клиентов. Для этого разделим доходы клиентов на 4 равные в процентном соотношении группы. Для применим метод *qcut()*.

In [30]:
df['income_group'] = pd.qcut(df['total_income'], q = [0, 0.25, 0.5, 0.75, 1], 
                             labels = ['низкий доход', 'средний доход', 'высокий доход', 'очень высокий доход'])
df['income_group'].head(10)

0    очень высокий доход
1          средний доход
2          средний доход
3    очень высокий доход
4          высокий доход
5    очень высокий доход
6    очень высокий доход
7          средний доход
8           низкий доход
9          средний доход
Name: income_group, dtype: category
Categories (4, object): [низкий доход < средний доход < высокий доход < очень высокий доход]

**Вывод**

Были созданы "словари", с помощью которых будет проще ответить на заданные вопросы. Также,"словари" помогают ориентироваться по таблице.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на этот вопрос нужно посчитать конверсию для каждой группы количества детей. В столбце *debt* значение *0* показывает, что у этого клиента не было задолжностей по кредиту, а значение *1*, что задолжность была. Чтобы посчитать конверсию надо в каждой группе поделить сумму *0* и *1* в столбце *debt* на их количество. Чем меньше будет конверсия, тем ответственнее клиенты в соответственной группе относятся к кредиту. 

In [35]:
debt_children = pd.DataFrame()

debt_children['sum'] = df.groupby('children')['debt'].sum()
debt_children['count'] = df.groupby('children')['debt'].count()
debt_children['conversion_in_percent'] = (debt_children['sum'] / debt_children['count']) * 100
debt_children = debt_children.sort_values(by = 'conversion_in_percent')

display(debt_children)

children_pivot = df.pivot_table(index = 'children',values = 'debt', aggfunc = ['mean'])
children_pivot

,sum,count,conversion_in_percent
children,,,
5,0,9,0.000000
0,1071,14167,7.559822
3,27,330,8.181818
1,445,4855,9.165808
2,194,2052,9.454191
4,4,41,9.756098


,mean
,debt
children,
0,0.075598
1,0.091658
2,0.094542
3,0.081818
4,0.097561
5,0.000000


**Вывод**

После группировки по количеству детей образовалась уже иная картина. Все также видна зависимость "больше детей - больше задолжностей", но из нее выпадают клиенты с тремя и пятью детьми. Вероятно, так получилось из-за сравнительного малого количества клиентов с такими показателями. Еще именно по этой причине клиенты с четырмя детьми стали "рекордсменами" по проценту задолжностей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Чтобы ответить на этот вопрос проделаем то же самое, что и в предыдущем вопросе, только подсчет конверсии будет по группам семейного положения.

In [32]:
debt_family = pd.DataFrame()

debt_family['sum'] = df.groupby('family_status')['debt'].sum()
debt_family['count'] = df.groupby('family_status')['debt'].count()
debt_family['conversion_in_percent'] = (debt_family['sum'] / debt_family['count']) * 100
debt_family = debt_family.sort_values(by = 'conversion_in_percent')

print(debt_family)

family_pivot = df.pivot_table(index = 'family_status',values = 'debt', aggfunc = ['mean'])
display(family_pivot)

                       sum  count  conversion_in_percent
family_status                                           
вдовец / вдова          63    959               6.569343
в разводе               85   1195               7.112971
женат / замужем        931  12339               7.545182
гражданский брак       388   4151               9.347145
Не женат / не замужем  274   2810               9.750890


,mean
,debt
family_status,
Не женат / не замужем,0.097509
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093471
женат / замужем,0.075452


**Вывод**

Самыми ответственными плательщиками оказались овдовевшие клиенты, недалеко ушли клиенты пережившие развод и официально зарегистрировавшие свой брак клиенты. Клиенты живущие в гражданском браке и неженатые/незамужние клиенты имеют значительный отрыв от других категорий в плане возврата кредита, что показывает их малый опыт не только в семейной жизни, но и в обращении с деньгами. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Аналогичным способом посчитаем конверсию для каждого типа дохода.

In [33]:
debt_income = pd.DataFrame()

debt_income['sum'] = df.groupby('income_group')['debt'].sum()
debt_income['count'] = df.groupby('income_group')['debt'].count()
debt_income['conversion_in_percent'] = (debt_income['sum'] / debt_income['count']) * 100
debt_income = debt_income.sort_values(by = 'conversion_in_percent')

print(debt_income)

income_pivot = df.pivot_table(index = 'income_group',values = 'debt', aggfunc = ['mean'])
income_pivot

                     sum  count  conversion_in_percent
income_group                                          
очень высокий доход  345   4909               7.027908
низкий доход         427   5364               7.960477
высокий доход        494   5818               8.490890
средний доход        475   5363               8.856983


,mean
,debt
income_group,
низкий доход,0.079605
средний доход,0.088570
высокий доход,0.084909
очень высокий доход,0.070279


**Вывод**

Клиенты с очень высоким доходом относятся к кредитам более ответственно, чем остальные клиенты в остальных группах. Примечательно, что на втором месте расположились клиенты с низким доходом, что показывает их более надежными плательщиками в сравнении с клиентами с высокими и средними доходами.

- Как разные цели кредита влияют на его возврат в срок?

В этом вопросе сгруппируем конверсию по обощенным целям кредита.

In [34]:
debt_purpose = pd.DataFrame()

debt_purpose['sum'] = df.groupby('common_purpose')['debt'].sum()
debt_purpose['count'] = df.groupby('common_purpose')['debt'].count()
debt_purpose['conversion_in_percent'] = (debt_purpose['sum'] / debt_purpose['count']) * 100
debt_purpose = debt_purpose.sort_values(by = 'conversion_in_percent')

print(debt_purpose)

purpose_pivot = df.pivot_table(index = 'common_purpose',values = 'debt', aggfunc = ['mean'])
purpose_pivot

                sum  count  conversion_in_percent
common_purpose                                   
недвижимость    782  10811               7.233373
свадьба         186   2324               8.003442
образование     370   4013               9.220035
автомобиль      403   4306               9.359034


,mean
,debt
common_purpose,
автомобиль,0.093590
недвижимость,0.072334
образование,0.092200
свадьба,0.080034


**Вывод**

Самой приоритетной целью для закрытия кредита оказалась недвижимость и все, что с ней связано. На втором месте, как ни странно, оказалась свадьба. Видимо денежные подарки на свадьбу все же сыграли свою роль. На последних местах, с небольшим отрывом, оказались цели, связанные с образованием и авто.

## Шаг 4. Общий вывод

В ходе выполнения проекта данные были очищены от пропусков и дубликатов для их подготовке к анализу. Была найдена причина в отрицательных и слишком больших положительных значениях трудового стажа. Проведена лемматизация столбца с целью кредита для более обобщенной категоризации данных. Были составлены "словари" для более эффективного ориентирования по таблице. 

Каждый вопрос нашел свой ответ с показательной зависимостью. Чем больше детей у клиентов, тем больше они не выплачивают кредиты в срок. Клиенты, состоющие в браке или имевшие опыт семейной жизни, более ответственно относятся к выплате кредита. Клиенты с очень высоким и низким доходами более рационально подходят к выбору суммы кредита, которая им по силе. Клиенты быстрее всего пытаются рассчитаться с кредитами, взятыми на недвижимость и свадьбу.